# Colab Notebook RAG Assistant

Short Notebook Description:

This Jupyter/Colab notebook builds a Retrieval-Augmented Generation (RAG) assistant over your own collection of .ipynb files in Google Colab. It:

1. Loads all notebooks from a local folder or mounted Google Drive.

2. Chunks their content into manageable pieces.

3. Embeds each chunk with OpenAI embeddings and stores them in a persistent Chroma vector database.

4. Provides a ConversationalRetrievalChain with memory and a Gradio chat interface.

5. For any user question, it returns both an answer and the names of the exact notebooks where the relevant information was found.

This setup lets you query your entire notebook history—whether local or in Colab—just like a personal knowledge base.

## Imports

In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import DirectoryLoader, NotebookLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA

from sklearn.manifold import TSNE
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np

## Configuration & Set Environment

In [ ]:
load_dotenv(override = True)

OPENAI_KEY = os.getenv('OPENAI_API_KEY')
NOTEBOOKS_DIR = os.getenv('NOTEBOOKS_DIR')
VECTOR_DB_DIR = os.getenv('VECTOR_DB_DIR')

In [ ]:
MODEL = 'gpt-4o-mini'
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [ ]:
try:
    # Colab
    from google.colab import drive
    print("Running in Colab: mounting Google Drive...")
    drive.mount('/content/drive')
    is_colab = True

    # Colab defaults
    NOTEBOOKS_DIR = '/content/drive/MyDrive/ColabNotebooks'
    DB_DIR = VECTOR_DB_DIR or '/content/drive/MyDrive/colab_vector_db'

except ImportError:
    # Local Jupyter Lab:
    print("Not in Colab: using local notebooks directory.")
    NOTEBOOKS_DIR = os.path.expanduser(NOTEBOOKS_DIR)
    DB_DIR = VECTOR_DB_DIR

    # Verify the local notebooks directory exists
    if not os.path.isdir(NOTEBOOKS_DIR):
        raise FileNotFoundError(
            f"Local notebooks directory '{NOTEBOOKS_DIR}' not found." 
            "\nPlease sync your Google Drive folder (e.g., via Drive for Desktop) "
            "or set NOTEBOOKS_DIR in your .env to the correct path."
        )
# Confirm final paths
# print(f"Indexing notebooks from: {NOTEBOOKS_DIR}")
# print(f"Chroma will store embeddings in: {DB_DIR}")

## Read in Notebook files

In [ ]:
notebooks = glob.glob(
    os.path.join(NOTEBOOKS_DIR, "**", "*.ipynb"),
    recursive=True
)
print(f"Notebooks found: {len(notebooks)}")


loader = DirectoryLoader(NOTEBOOKS_DIR,
                         glob = '**/*.ipynb',
                         loader_cls = NotebookLoader
                        )
docs = loader.load()

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = CHUNK_SIZE, 
    chunk_overlap = CHUNK_OVERLAP, 
    separators=["\n## ", "\n### ", "\n#### ", "\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(f'Created {len(chunks)} chunks from your notebooks')

## Embedding

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

if os.path.exists(DB_DIR):
    Chroma(persist_directory = DB_DIR, embedding_function = embeddings).delete_collection()


vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings,
    persist_directory = VECTOR_DB_DIR
)

vector_count = vectorstore._collection.count()
print(f"Vectorstore contains {vector_count} vectors.")

In [ ]:
sample_embedding = vectorstore._collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {vectorstore._collection.count():,} vectors with {dimensions:,} dimensions in the vector store.")

## Visualize in 3D

In [ ]:
result = vectorstore._collection.get(include=['embeddings', 'documents',])
vectors = np.array(result['embeddings'])
documents = result['documents']
colors = ['blue'] * len(vectors)

In [ ]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.8),
    text=[d[:100] + "..." for d in documents],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D TSNE of Notebook-Chunks',
    scene=dict(
        xaxis_title="TSNE-1",
        yaxis_title="TSNE-2",
        zaxis_title="TSNE-3"
    ),
    width=800,
    height=600,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

## Build LLM

In [ ]:
llm = ChatOpenAI(model_name = MODEL, temperature = 0.6)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True
)

In [ ]:
memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    return_messages = True
)

conv_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = vectorstore.as_retriever(search_kwargs = {'k': 10}),
    memory = memory
)

In [ ]:
def chat_with_memory_and_sources(message, chat_history):
    # Get the conversational answer (memory included)
    conv_res = conv_chain.invoke({
        "question":     message,
        "chat_history": chat_history
    })
    answer = conv_res["answer"]

    # Retrieve source documents  
    src_res  = qa({"query": message})
    src_docs = src_res["source_documents"]

    # Extract and dedupe notebook filenames from metadata
    notebooks = [
        os.path.basename(doc.metadata.get("source", ""))
        for doc in src_docs
        if doc.metadata.get("source")
    ]
    unique = []
    for n in notebooks:
        if n not in unique:
            unique.append(n)

    # Append the list of notebook filenames
    if unique:
        answer += "\n\n**Found Notebooks:**\n" + "\n".join(f"- {n}" for n in unique)
    else:
        answer += "\n\n_No Notebooks found._"

    return answer

## Gradio UI

In [ ]:
view = gr.ChatInterface(chat_with_memory_and_sources, 
                        title="Notebook-RAG-Assistant mit Memory & Quellen",
                       type = 'messages').launch(inbrowser=True)